In [1]:
import pandas as pd
import pickle
import make_question
import ranking_help_methods

from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

In [2]:
dataset = pd.read_csv('processed_perPage_perLine.csv')
dataset = dataset.iloc[:,1:]
dataset.head()

,Data,PageID
0,カスタマイザー タイトル 設定 カスタマイザー サブタイトル 設定 MobiControl ...,0
1,Apple 製品 MobiControl v ManualApple 製品 MobiCont...,1
2,Android 端末 MobiControl v ManualAndroid 設定 モード ...,2
3,Windows PCMobiControl v ManualWindows PC 根本 セキ...,3
4,Windows EmbeddedMobiControl v ManualWindows Em...,4


In [3]:
corpus_text = ' '.join(dataset.Data)
corpus_list = corpus_text.split()
words_list = list(set(corpus_list))
word_ids = list(enumerate(words_list))
word_ids = sorted(word_ids, key=lambda l:l[1], reverse=True)

# i should be the first vector followed by 受け, Base, 製造元
word_ids

[(5802, '￥'),
 (4329, 'ｷｬﾝｾﾙ'),
 (5722, '･'),
 (4387, '｣'),
 (6317, '｢'),
 (138, '～'),
 (5379, '｝'),
 (1925, '｜'),
 (3661, 'ｔ'),
 (2843, 'ｍ'),
 (5969, 'ｋ'),
 (5346, 'ｄ'),
 (3258, '＠'),
 (904, '？'),
 (4936, '＝'),
 (2272, '：%'),
 (1190, '：'),
 (5863, '．'),
 (5362, '－'),
 (809, '％'),
 (7044, '！'),
 (1467, '黒鉛'),
 (1852, '黒色'),
 (6886, '黒い'),
 (6272, '黒'),
 (2042, '黄色い'),
 (3823, '黄色'),
 (6776, '鳴る'),
 (3797, '鳴り'),
 (5010, '鳴らす'),
 (1235, '鳴らし'),
 (6465, '高額'),
 (1982, '高速'),
 (5673, '高度'),
 (5457, '高める'),
 (1608, '高め'),
 (4899, '高まり'),
 (1480, '高く'),
 (6798, '高い'),
 (3763, '高'),
 (4828, '駆除'),
 (1299, '駆動'),
 (1992, '飾り'),
 (4657, '飲食'),
 (6780, '食堂'),
 (3581, '風車'),
 (6162, '風'),
 (5504, '顧客'),
 (3097, '類推'),
 (6546, '類似'),
 (1745, '類'),
 (3006, '願い'),
 (5997, '顕著'),
 (4452, '顔'),
 (72, '額'),
 (4962, '頻繁'),
 (3257, '頻度'),
 (5280, '領域'),
 (5649, '順番'),
 (6994, '順次'),
 (4843, '順序'),
 (71, '順位'),
 (1338, '順に'),
 (1322, '順'),
 (5293, '項目'),
 (841, '項'),
 (2464, '頃'),
 (358, '頂点'),
 (2788, '

In [4]:
# Vectorizer Just Words -> IDs
def custom_vectorizer(input_list):
    collector = []
    for token in input_list:
        for key, value in word_ids: 
            if value == token:
                # print(value, "->", key)
                collector.append(key)
                break
    return collector

text_vectors=[]
page_ids = []
for index, col in dataset.iterrows():
    text_vectors.append(custom_vectorizer(str(col['Data']).split()))
    page_ids.append(col['PageID'])

vector = list(zip(text_vectors, page_ids))



# Not writing on disk because already it is present there
with open('vectormodel_perpage_v1.pkl', 'wb') as f:
    pickle.dump(vector ,f)
    
with open('vectormodel_perpage_v1.pkl', 'rb') as f:
    vector = pickle.load(f)
    
len(vector)    

445

In [5]:

questions_corpus = make_question.make_question(make_question.split_lines_corpus(dataset))


In [8]:
sample_size = 100
questions_samples = questions_corpus.head(int(sample_size))

questions_samples.head()

,Question,PageID
0,Tokyo Penetrate Ltd Manual MobiControl v Limit...,254
1,生徒 構成 macOS できる ユーザー URL 時間 制限 アクセス どう,296
2,すぐ スケジュール なく 割り当てる 適用 ずらす 端末 グループ 上位 開発を開始する場所,148
3,おき アラート MobiControl 値 なっ 特定 扱う た 認識 どちら,412
4,おく い アラートアクション チェック 繰り返し 端末 執る 入れ オプション どちら,43


In [9]:

def get_score_details_record(dataset, questions_samples, vectorizer, vector, gensim_active=False):
    sample_count = 0
    sum_score = 0
    container = []

    for index, col in questions_samples.iterrows():
        query = str(col['Question'])
        input_vector = list(set(str(query).split()))
        query_vector = custom_vectorizer(input_vector)
        
        rank_list = ranking_help_methods.custom_countvectorizer_ranking(dataset, vector, query, 
                                                                        query_vector, vectorizer, gensim_active)
        #print(rank_list)
        
        page_answers = []
        prediction_scores = []
        for ids, score in rank_list:
            # print(ids, score)
            page_answers.append(ids)
            prediction_scores.append(score)

        MRR_score = ranking_help_methods.mean_reciprocal_rank_score(col['PageID'], page_answers)
        sum_score += MRR_score

        container.append([MRR_score, col['PageID'], page_answers, prediction_scores, col['Question']])
        sample_count += 1
        
    result = pd.DataFrame(container, columns=['score', 'actual_answer', 'page_answers', 'prediction_scores',
                                              'query'])
    result.to_csv('performance.csv')
    score = sum_score / sample_count

    return score


ranks = get_score_details_record(dataset, questions_samples, vectorizer, vector, False)
ranks

0.4794999999999998

# Experiments Starts from below
